# Software Coaching for Python
# Week 5: Document Clustering & Topic Modeling - Homework

Instructor: Kang-Pyo Lee

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = "/content/gdrive/Shareddrives/Software_Coaching_Fall_2021"
my_folder = "KPL"     # *** REPLACE WITH YOUR FOLDER NAME ***
outcome_folder = f"{path}/{my_folder}/outcome"
classdata_folder = f"{path}/classdata"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! pip install --user scikit-learn pyldavis

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

df = pd.read_csv(f"{classdata_folder}/timeline_NASA.csv", sep="\t")
df.text = df.text.astype(str)
df

Check if there are any duplicates in the `text` column.

In [ ]:
df.text.value_counts()

Remove duplicates in the `text` column, if any. 

In [ ]:
df = df.drop_duplicates(["text"], keep="first").copy()

In [ ]:
df.text.value_counts()

In [ ]:
df.info()

## Preparing Data for Modeling

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])

import string 
global_stopwords = stopwords.words("english")

Define your own local stopwords. 

In [ ]:
local_stopwords = [c for c in string.punctuation] +\
                  ['‘', '’', '—', '…', '“', '”', "'re", "'s", '...', 'a…', 't…', 'https', "n't", 'rt']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm="l2", stop_words=global_stopwords+local_stopwords, max_df=0.7)
X = vectorizer.fit_transform(df.text)

In [ ]:
X.shape

## K-Means Clustering

### Step 1. Choose the number of clusters

Set the number of clusters you would like to get. 

In [ ]:
# Your answer here
k = 5

### Step 2. Initialize a model object for k-means clustering

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, random_state=0)
kmeans

### Step 3. Fit the model using the input data

In [ ]:
%time kmeans.fit(X)

### Step 4. Examine the clustering outcome

In [ ]:
kmeans.labels_

In [ ]:
df["cluster"] = kmeans.labels_

In [ ]:
df[["text", "cluster"]]

Check the number of values for each cluster. 

In [ ]:
df.cluster.value_counts()

Check 10 random texts from the largest cluster. 

In [ ]:
df[df.cluster == 1].sample(10, random_state=0)[["text", "cluster"]]

Check 10 random texts from the second largest cluster. 

In [ ]:
df[df.cluster == 2].sample(10, random_state=0)[["text", "cluster"]]

In [ ]:
import nltk
df["words"] = df.text.apply(lambda x: nltk.word_tokenize(x))
df["tagged_words"] = df.words.apply(lambda x: nltk.pos_tag(x))

from collections import Counter

def get_counter(dataframe, stopwords=[]):
    counter = Counter()
    
    for l in dataframe.tagged_words:
        word_set = set()

        for t in l:
            word = t[0].lower()
            tag = t[1]

            if word not in stopwords:
                word_set.add(word)
            
        counter.update(word_set)
        
    return counter

Check the top-30 most common keywords in the largest cluster. (Replace the cluster number as needed.)

In [ ]:
counter0 = get_counter(df[df.cluster == 1], global_stopwords+local_stopwords)
counter0.most_common(30)

Check the top-30 most common keywords in the second largest cluster. (Replace the cluster number as needed.)

In [ ]:
counter4 = get_counter(df[df.cluster == 2], global_stopwords+local_stopwords)
counter4.most_common(30)

## LDA Topic Modeling

### Step 1. Choose the number of topics

Set the number of topics you would like to get. 

In [ ]:
# Your answer here
num_topics = 5

### Step 2. Initialize a model object for LDA topic modeling

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=num_topics, random_state=0)
lda

### Step 3. Fit the model using the input data

In [ ]:
%time lda.fit(X)

### Step 4. Examine the output of topic modeling

In [ ]:
def show_topics(model, feature_names, num_top_words):
    for topic_idx, topic_scores in enumerate(model.components_):
        print("***Topic {}:".format(topic_idx))
        print(" + ".join(["{:.2f} * {}".format(topic_scores[i], feature_names[i]) for i in topic_scores.argsort()[::-1][:num_top_words]]))
        print()

In [ ]:
show_topics(lda, vectorizer.get_feature_names(), 10)

### Topic Model Visualization

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.sklearn.prepare(lda, X, vectorizer)